**Giải thuật Salsa20/12**

In [10]:
def rotate_left(value, shift, bits=32):
    """Rotate left operation on a 32-bit integer value."""
    return ((value << shift) | (value >> (bits - shift))) & ((1 << bits) - 1)

def quarter_round(a, b, c, d):
    """Perform a quarter round operation on four 32-bit integers."""
    a = (a + b) & 0xffffffff
    d ^= a
    d = rotate_left(d, 16)

    c = (c + d) & 0xffffffff
    b ^= c
    b = rotate_left(b, 12)

    a = (a + b) & 0xffffffff
    d ^= a
    d = rotate_left(d, 8)

    c = (c + d) & 0xffffffff
    b ^= c
    b = rotate_left(b, 7)

    return a, b, c, d

def salsa2012_hash(state):
    """Compute the Salsa20/12 hash function on a 512-bit state."""
    x = list(state)
    for _ in range(6):  # 12 rounds, performed in pairs
        # Column rounds
        x[4], x[8], x[12], x[0] = quarter_round(x[4], x[8], x[12], x[0])
        x[9], x[13], x[1], x[5] = quarter_round(x[9], x[13], x[1], x[5])
        x[14], x[2], x[6], x[10] = quarter_round(x[14], x[2], x[6], x[10])
        x[3], x[7], x[11], x[15] = quarter_round(x[3], x[7], x[11], x[15])
        # Diagonal rounds
        x[1], x[2], x[3], x[0] = quarter_round(x[1], x[2], x[3], x[0])
        x[6], x[7], x[4], x[5] = quarter_round(x[6], x[7], x[4], x[5])
        x[11], x[8], x[9], x[10] = quarter_round(x[11], x[8], x[9], x[10])
        x[12], x[13], x[14], x[15] = quarter_round(x[12], x[13], x[14], x[15])
    return [(x[i] + state[i]) & 0xffffffff for i in range(16)]

def generate_key_schedule(key, nonce, counter):
    """Generate the Salsa20 key schedule."""
    constants = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]
    key_schedule = constants[:1] + [int.from_bytes(key[i:i+4], 'little') for i in range(0, 16, 4)] \
                  + constants[1:3] + [counter & 0xffffffff, counter >> 32] \
                  + [int.from_bytes(nonce[i:i+4], 'little') for i in range(0, 8, 4)] + constants[3:]
    if len(key) == 32:
        key_schedule[1:5] += [int.from_bytes(key[i:i+4], 'little') for i in range(16, 32, 4)]
    elif len(key) != 16:
        raise ValueError("Key must be either 16 or 32 bytes long.")
    return key_schedule

def encrypt(plaintext, key, nonce):
    """Encrypt plaintext using the Salsa20 algorithm."""
    assert len(nonce) == 8, "Nonce must be 8 bytes long"
    encrypted = b''
    counter = 0
    while plaintext:
        block = plaintext[:64]
        plaintext = plaintext[64:]
        key_schedule = generate_key_schedule(key, nonce, counter)
        keystream = salsa20_hash(key_schedule)
        keystream_bytes = b''.join(word.to_bytes(4, 'little') for word in keystream)
        encrypted_block = bytes(a ^ b for a, b in zip(block, keystream_bytes[:len(block)]))
        encrypted += encrypted_block
        counter += 1
    return encrypted

def decrypt(ciphertext, key, nonce):
    """Decrypt ciphertext using the Salsa20 algorithm."""
    return encrypt(ciphertext, key, nonce)

# Example usage
key = b"Today_I_code_Salsa2012_algorithm"
nonce = b"mynonce1"
plaintext = b"Congatulation, you remake Salsa2012"

encrypted = encrypt(plaintext, key, nonce)
print(f"Encrypted: {encrypted.hex()}")

decrypted = decrypt(encrypted, key, nonce)
print(f"Decrypted: {decrypted.decode()}")


Encrypted: eb5eadb64f763f988da746d1c834cc5627ff07353fe376b9de41da81b36cd029a6b8e5
Decrypted: Congatulation, you remake Salsa2012


In [11]:
def key_scheduling_algorithm(key):
    key_length = len(key)
    S = list(range(256))
    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % key_length]) % 256
        S[i], S[j] = S[j], S[i]
    return S

def pseudo_random_generation_algorithm(S):
    i = 0
    j = 0
    while True:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        K = S[(S[i] + S[j]) % 256]
        yield K

def RC4_algorithm(key):
    S = key_scheduling_algorithm(key)
    return pseudo_random_generation_algorithm(S)

In [14]:
import time

def benchmark_stream_generation(cipher_func, encryption_key, nonce=None, data_size=1024*1024):
    # Bắt đầu đo thời gian
    start_time = time.time()

    # Tạo một khối dữ liệu
    data_block = b'\x00' * data_size

    # Kiểm tra sự tồn tại của nonce để xác định thuật toán nào đang được sử dụng
    if nonce:
        # Đối với Salsa20/12
        _ = cipher_func(data_block, encryption_key, nonce)
    else:
        # Đối với RC4, giả định rằng cipher_func được điều chỉnh để trả về dữ liệu dưới dạng bytes
        _ = cipher_func(encryption_key, data_block)

    # Kết thúc đo thời gian và tính toán thời gian chạy
    end_time = time.time()
    generation_time = end_time - start_time

    return generation_time

# Điều chỉnh RC4 để trả về dữ liệu dưới dạng bytes thay vì một generator
def RC4_to_bytes(encryption_key, data_block):
    keystream = RC4(encryption_key)
    return bytes([next(keystream) for _ in range(len(data_block))])

# Đo thời gian sinh dữ liệu cho Salsa20 và RC4
salsa2012_generation_time = benchmark_stream_generation(encrypt, key, nonce)
rc4_generation_time = benchmark_stream_generation(RC4_to_bytes, key)

print(f"Salsa20/12 Generation Time: {salsa2012_generation_time} seconds")
print(f"RC4 Generation Time: {rc4_generation_time} seconds")


Salsa20/12 Generation Time: 4.940304756164551 seconds
RC4 Generation Time: 0.4647209644317627 seconds


In [17]:
def key_sensitivity_test(encrypt_function, encryption_key, nonce=None):
    # Tạo một khối dữ liệu nhỏ để mã hóa
    data_block = b'\x00' * 512

    # Tạo một bản sao của key với bit đầu tiên bị đảo
    flipped_key = bytearray(encryption_key)
    flipped_key[0] ^= 1

    # Mã hóa dữ liệu với key ban đầu và key đã được đảo bit
    encrypted_data1 = encrypt_function(data_block, encryption_key, nonce) if nonce else RC4_to_bytes(encryption_key, data_block)
    encrypted_data2 = encrypt_function(data_block, bytes(flipped_key), nonce) if nonce else RC4_to_bytes(bytes(flipped_key), data_block)

    # Tính số lượng byte khác nhau giữa hai dữ liệu đã mã hóa
    differing_bytes_count = sum(1 for x, y in zip(encrypted_data1, encrypted_data2) if x != y)
    return differing_bytes_count

# Thực hiện thử nghiệm so sánh độ nhạy khóa
difference_salsa2012 = key_sensitivity_test(encrypt, key, nonce)
difference_rc4 = key_sensitivity_test(RC4_to_bytes, key)

print(f"Sensitivity of Salsa20/12 to Key: {difference_salsa2012} bytes differ")
print(f"Sensitivity of RC4 to Key: {difference_rc4} bytes differ")


Sensitivity of Salsa20/12 to Key: 510 bytes differ
Sensitivity of RC4 to Key: 509 bytes differ
